In [3]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

### Modelo con asimetría de información|

El votante representativo se preocupa por el valor esperado de su función de utilidad  $\mathbb{E}_{t}^{P}(\Gamma_{t})$

<ul>
    <li>c representa el consumo del bien privado por parte de un ciudadano representativo.</li>
    <li>g es el consumo del bien público (per cápita)</li>
    <li>k es la inversión pública (per cápita)</li>
</ul>

$$\begin{equation}
    \begin{aligned}
        \Gamma_{t}=\sum_{s=t}^{T}[U(c_{s}, g_{s}) + V(k_{s}) + \eta_{s}]\beta^{s-t}
    \end{aligned}
    \end{equation}$$

$$c_{t} = y - \tau_{t}$$
$$g_{t} +k_{t+1} = \tau_{t} + \epsilon_{t}$$

$$\begin{equation}
    \begin{aligned}
        W(g, \tau, \epsilon)=U(y-\tau, g) + \beta V(\tau + \epsilon - g)
    \end{aligned}
    \end{equation}$$

Si los votantes observan el último shock de competencia del incumbente antes de votar $\alpha_{i}$, entonces las expectativas hacia el incumbente serian.
$$ \mathbf{E}_{t}^{P}[W^{*}(\epsilon_{t+1})|\alpha_{t}=\alpha^{i}]=\Omega^{i}$$
Donde $i=H, L$ para los tipos compententes y no competentes en un modelo de senalizacion 

Para el caso del oponente, no se observa cuan competente es, entonces:
$$\mathbf{E}_{t}^{P}[W^{*}(\epsilon_{t+1}^{O})]=\Omega^{O}$$


El problema de maximizacion del incumbente de tipo $i=H, L$ seria:
$$\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & \beta[X(1+\beta) + \Omega^{i}-\Omega^{O}]\pi[\hat{\rho}(g,\tau)]+ W(g, \tau, \epsilon^{i}) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon^{i}-g \geq 0 \\
    \end{aligned}
    \end{equation}$$

Para la probabilidad de ganar elegimos una función que toma valores entre 0 y 1 con una relación positiva en g y una negativa en tau para reflejar las preferencias de los agentes por bajos impuestos y alto consumo público, al hacer esto le permitimos al agente manipular $\tau$ y $g$ pero al no modelar el valor esperado de la funcion de votos estariamos tambien quitandole cierta logica al agente racional que no estaria comparando sus utilidades esperadas con el incumbente y con el nuevo.

$$\begin{equation}
    \begin{aligned}
        \pi[\hat{\rho}(\tau, g)] = \mathbf{E}^{I}(v|g, \tau)
    \end{aligned}
    \end{equation}$$

Función de elección del líder

\begin{equation}
    v_{t} =
    \begin{cases}
        1 &  \text{si } \mathbf{E}_{t}^{P}(\Gamma_{t+1}) \geq \mathbf{E}_{t}^{P}(\Gamma_{t+1}^{O})\\
        0 & \text{en otro caso}\\
\end{cases}
\end{equation}

____________________

## Definiciones 

$$\begin{equation}
    \begin{aligned}
        \pi[\hat{\rho}(\tau, g)] = \frac{1}{1+e^{dg-h\tau+\eta_{t}^{i}}}
    \end{aligned}
    \end{equation}$$

Donde: $d$, $h$ y $m$ son valores positivos que controlan la sensibilidad de la funcion logistica elegida, calibrar estos valores para reflejar la probabilidad de ganar del incumbente tambien sera un ejercicio clave que podria hacerse a traves de las encuestas, la funcion $\Gamma$ debe ser utilizada para determinar la funcion de votacion comparando esas utilidades esperadas, solo de esa manera se inserta dentro la probabilidad, igual se podria usar esta probabilidad

$$ \epsilon_{t}^{i} = \alpha_{t}^{i} + \alpha_{t-1}^{i}$$
$$ \eta_{t}^{i} = q_{t}^{i} + q_{t-1}^{i}$$

$$U(c, g) = a \ln{c} + b\ln{g}$$
$$V(k) = -\ln{k}$$

### Parámetros

In [4]:
a, b = 1, 1     # constantes de la función de utilidad U
beta = 0.95     # Tasa de descuento intertemporal
X = 1           # Rentas de ego provenientes de ser líder

y = 100         # Producto en unidades de un bien no almacenable

T = 10          # Número máximo de periodos
t = 0           # Periodo inicial 

### Calibración de variables aleatorias

Se eligen estos valores tanto para el incumbente como para su oponente

In [2]:
p = 0.5                 # Probabilidad de que el agente i sea competente en el periodo t
q_r = range(-10,10)     # Rango de valores donde viven los shocks de apariencia iguales para el incumbente y para el oponente

In [13]:
alpha1_i = np.random.binomial(1, p, T-t)    # Asumimos una distribución binomial para la competencia del incumbente
alpha2_i = np.random.binomial(1, p, T-t)    # Asumimos una distribución binomial para la competencia del incumbente

alpha1_o = np.random.binomial(1, p, T-t)    # Asumimos una distribución binomial para la competencia del oponente
alpha2_o = np.random.binomial(1, p, T-t)    # Asumimos una distribución binomial para la competencia del oponente

epsilon_i = alpha1_i + alpha2_i             # Competencia del Incumbente independiente entre agentes y en el tiempo
epsilon_o = alpha1_o + alpha2_o             # Competencia del Oponente independiente entre agentes y en el tiempo

# -------------------------------------------------------------------------------------------------------------------
q1_i = np.random.choice(q_r, T-t)           # Los shocks de apariencia se eligen de forma aleatoria entre el rango q_r
q2_i = np.random.choice(q_r, T-t)           # Los shocks de apariencia se eligen de forma aleatoria entre el rango q_r

q1_o = np.random.choice(q_r, T-t)           # Los shocks de apariencia se eligen de forma aleatoria entre el rango q_r
q2_o = np.random.choice(q_r, T-t)           # Los shocks de apariencia se eligen de forma aleatoria entre el rango q_r

eta_i = q1_i + q2_i                         # Shock no relacionado con la competencia pero si con el liderazgo del Incumbente independiente entre agentes y en el tiempo
eta_o = q1_o + q2_o                         # Shock no relacionado con la competencia pero si con el liderazgo del Oponente independiente entre agentes y en el tiempo

### Funciones

Definimos las funciones que vamos a usar 

$$\begin{equation}
    \begin{aligned}
        W(g, \tau, \epsilon)=U(y-\tau, g) + \beta V(\tau + \epsilon - g)
    \end{aligned}
    \end{equation}$$

In [15]:
def W(x, y, epsilon, beta, a, b):
    tau, g = x
    c = y - tau
    k = tau + epsilon - g
    return -(a * np.log(c) + b * np.log(g) + beta * (-np.log(k)))  # Negativo porque usamos un minimizador de scipy

$$\begin{equation}
    \begin{aligned}
        \Gamma_{t}=\sum_{s=t}^{T}[U(c_{s}, g_{s}) + V(k_{s}) + \eta_{s}]\beta^{s-t}
    \end{aligned}
    \end{equation}$$

In [16]:
def gamma(x, y, eta, beta, epsilon, t, T, a, b):
    tau, g = x
    c = y - tau
    k = tau + epsilon - g
    return sum([(a * np.log(c) + b * np.log(g) + (-np.log(k[s])) + eta[s])*beta**(s-t) for s in range(t, T)])

$$\begin{equation}
    \begin{aligned}
        \pi[\hat{\rho}(\tau, g)] = \frac{1}{1+e^{dg-h\tau+\eta_{t}^{i}}}
    \end{aligned}
    \end{equation}$$

In [17]:
def pi(x, d, h, eta_i):
    tau, g = x
    return 1/(1+np.exp(d*g-h*tau+eta_i[t]))

Definimos una función que otmiza la función W para cada periodo de tiempo, ya que será usada como insumo en la función principal del modelo

$$\begin{equation}
    \begin{aligned}
        \mathbf{E}_{t}^{P}[W^{*}(\epsilon_{t+1})|\alpha_{t}=\alpha^{i}]=\Omega^{i}
        \end{aligned}
    \end{equation}$$
Con $i=H, L$ para los tipos con alta competencia o baja competencia respectivamente

$$\begin{equation}
    \begin{aligned}
        \mathbf{E}_{t}^{P}[W^{*}(\epsilon_{t+1}^{O})]=\Omega^{O}
        \end{aligned}
    \end{equation}$$

Donde $W^{*}(\epsilon)=W^{*}[g^{*}(\epsilon), \tau^{*}(\epsilon), \epsilon]$ viene de resolver
$$\begin{equation}
    \begin{aligned}
        \max_{\tau, g} & \beta[X(1+\beta) + \Omega^{i}-\Omega^{O}]\pi[\hat{\rho}(g,\tau)]+ W(g, \tau, \epsilon^{i}) \\
        \text{sujeto a } & g, y- \tau, \tau + \epsilon^{i}-g \geq 0 \\
    \end{aligned}
    \end{equation}$$

Para la maximización que el optimizador de scipy lo resuelve iterando la función de valor debemos proporcionar un valor inicial

In [23]:
# Guess inicial (Por ahora tau y g empiezan tomando valores en la mitad de su rango permitido)
x0 = [y/2, y/4]

In [24]:
def constraint1(x, y):
    # y - tau >= 0
    return y - x[0]

def constraint2(x):
    # g >= 0
    return x[1]

def constraint3(x, epsilon):
    # tau + epsilon - g >= 0
    return x[0] + epsilon - x[1]

def optimizador_W(T, y, beta, a, b, epsilon):
    tau_optimo = []
    g_optimo = []
    valores_max = []

    for t in range(T):
        # Itera sobre los valores de competencia del agente
        epsilon_t = epsilon[t]

        # Restricciones por cada periodo de tiempo(epsilon)
        con1 = {'type': 'ineq', 'fun': constraint1, 'args': (y,)}
        con2 = {'type': 'ineq', 'fun': constraint2}
        con3 = {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_t,)}

        # Combinamos las restricciones en una lista
        constraints_t = [con1, con2, con3]

        # Optimiza para ese periodo de tiempo t
        resultado_t = minimize(W, x0, args=(y, epsilon_t, beta, a, b), constraints=constraints_t)

        # Guarda los resultados para cada periodo de tiempo
        tau_optimo.append(resultado_t.x[0])
        g_optimo.append(resultado_t.x[1])
        valores_max.append(-resultado_t.fun)

    return tau_optimo, g_optimo, valores_max

Ahora si definimos la función del modelo con asimetría de información

In [26]:
def asimetria(x, y, a, b, t, T, beta, epsilon_i, epsilon_o, X, d, h, eta_i):
    lista_tau_i, lista_g_i, lista_utilidades_i = optimizador_W(T, y, beta, a, b, epsilon_i)
    lista_tau_o, lista_g_o, lista_utilidades_o = optimizador_W(T, y, beta, a, b, epsilon_o)
    valor_pi = pi(x, d, h, eta_i[t])  
    w_i = W(x, y, epsilon_i[t], beta, a, b)

    return -(beta*(X*(1+beta) + np.mean(lista_utilidades_i)-np.mean(lista_utilidades_o))*valor_pi + w_i) # Negativo para usar el minimizador de scipy

Resolvemos el problema de maximización del incumbente

In [32]:
asimetria(x0, 100, a, b, t, T, beta, epsilon_i[t], epsilon_o[t], 1, 0.5, 0.5, eta_i[t])

IndexError: invalid index to scalar variable.

In [34]:
args = (y, a, b, T, beta, epsilon_i, epsilon_o, 1, 0.5, 0.2, 4)     # Argumentos  de la función

# Definición de restricciones
constraints = (
    {'type': 'ineq', 'fun': constraint1, 'args': (y,)},
    {'type': 'ineq', 'fun': constraint2},
    {'type': 'ineq', 'fun': constraint3, 'args': (epsilon_i,)}  # Aquí epsilon_i es un escalar
)

# Realizar la optimización
resultado = minimize(asimetria, x0, args=args, constraints=constraints)

# Resultados óptimos
tau_optimo, g_optimo = resultado.x

TypeError: asimetria() missing 1 required positional argument: 'eta_i'